In [1]:
from SBM_attributed import *
import torch
import networkx as nx
from divergences import * 
from copy import deepcopy
from sklearn.metrics import rand_score, calinski_harabasz_score, mutual_info_score, accuracy_score

In [2]:
### Generate benchmark
P = np.array([[0.8, 0.2, 0.3],[0.2, 0.7, 0.4],[0.3, 0.4, 0.6]])
c = 3
n = 100
N = c*n
delta = 10
d = 2
dim = c*d
_,A,X = generate_benchmark(P,c,delta=delta,n=n,dim=d,sample_along_direction=True)
A = torch.tensor(A,dtype=torch.float32)
X = torch.tensor(X,dtype=torch.float32)

In [3]:
### initialize variables
W = torch.zeros((N,c),dtype=torch.float)
indexes = torch.randint(low=0,high=c,size=(N,))
with torch.no_grad():
    for index,col in enumerate(indexes):
        W[index].index_fill_(0, col, 1)
WW_inv = torch.inverse(W.T@W)
B = WW_inv@W.T@A@W@WW_inv
mu = WW_inv@W.T@X

In [4]:
phi_net = get_phi("euclidean",elementwise=True)[0]
phi_data = get_phi("euclidean")

In [5]:
def obj_func(W,B,mu):
    net_divergence = torch.sum(phi_net(A,W@B@W.T), axis=1)
    data_divergence = torch.sum(torch.multiply(W, pairwise_bregman(X, mu, phi_data)),axis=1)
    K = torch.stack((net_divergence,data_divergence),axis=-1)
    return K.sum()
#     return torch.linalg.matrix_norm(K,ord=-np.inf,keepdim=False)

In [6]:
classes_old = classes = None
convergence_cnt = 0
convergence_threshold = 5
iter = 0
failed_to_converge = False
while True:
        iter += 1
        for i in range(N):
            W[i,:] = 0
            min_ = torch.tensor(np.inf)
            min_index = torch.tensor(0)
            for j in torch.arange(c):
                W[i].index_fill_(0,j,1)
                val = obj_func(W,B,mu)
                if val < min_:
                    min_index = j
                    min_ = val
                W[i,:] = 0
            W[i].index_fill_(0,min_index,1)
        WW_inv = torch.inverse(W.T@W)
        B = WW_inv@W.T@A@W@WW_inv
        mu = WW_inv@W.T@X
        #update values
        if classes is not None:
            classes_old = classes
        classes = torch.argmax(W, axis=1)
        #Check convergence
        if classes_old is not None and classes is not None and torch.equal(classes_old, classes):
            convergence_cnt += 1
        else:
            convergence_cnt = 0
        if convergence_cnt == convergence_threshold:
            print("point assignments have converged")
            break

point assignments have converged


In [7]:
pred_labels = torch.argmax(W, axis=1).detach().numpy()
true_labels = [0]*n + [1]*n + [2]*n
print(rand_score(true_labels, pred_labels),mutual_info_score(true_labels, pred_labels))

1.0 1.0986122886681091


In [8]:
B

tensor([[0.7956, 0.2024, 0.2988],
        [0.2024, 0.6900, 0.4037],
        [0.2988, 0.4037, 0.6002]])

In [9]:
P

array([[0.8, 0.2, 0.3],
       [0.2, 0.7, 0.4],
       [0.3, 0.4, 0.6]])

In [10]:
mu

tensor([[-1.3504, -1.5721,  5.5747, -5.4138, -6.8401,  0.3553],
        [ 5.7807, -5.3352,  2.0913, -2.0637, -5.9000,  0.2521],
        [-7.0645,  0.4563, -5.9715,  0.2040, -3.9599,  0.2682]])

In [11]:
iter

7